In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import json

In [3]:
# Spark Constants
APP_NAME = 'assignment1'
MASTER = 'local[*]'

# Input Constants
INPUT_METADATA_FILE = 'tracks.csv'
INPUT_FEATURES_FILE = 'features.csv'

In [4]:
conf = SparkConf().setAppName(APP_NAME).setMaster(MASTER)
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession.builder.appName(APP_NAME).master(MASTER).getOrCreate()

23/04/13 09:17:14 WARN Utils: Your hostname, pedro-duarte resolves to a loopback address: 127.0.1.1; using 192.168.32.217 instead (on interface wlp2s0)
23/04/13 09:17:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 09:17:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [22]:
data = spark.read.csv(INPUT_METADATA_FILE, quote='"', escape='"', multiLine=True, inferSchema=True).rdd
headerRow = ['_'.join(header) if idx > 0 else 'track_id' for idx, header in enumerate(zip(*[list(line.asDict().values()) for line in data.take(2)]))]

headerRows = data.take(3)
data = data.filter(lambda row: row not in headerRows)

firstRow = data.first()

data = data \
  .map(lambda row: list(row.asDict().values())) \
  .map(lambda row: {k: v for k, v in zip(headerRow, row)}) \
  .map(json.dumps)

ds = spark.read.json(data, primitivesAsString=True)
ds.schema

StructType([StructField('album_comments', StringType(), True), StructField('album_date_created', StringType(), True), StructField('album_date_released', StringType(), True), StructField('album_engineer', StringType(), True), StructField('album_favorites', StringType(), True), StructField('album_id', StringType(), True), StructField('album_information', StringType(), True), StructField('album_listens', StringType(), True), StructField('album_producer', StringType(), True), StructField('album_tags', StringType(), True), StructField('album_title', StringType(), True), StructField('album_tracks', StringType(), True), StructField('album_type', StringType(), True), StructField('artist_active_year_begin', StringType(), True), StructField('artist_active_year_end', StringType(), True), StructField('artist_associated_labels', StringType(), True), StructField('artist_bio', StringType(), True), StructField('artist_comments', StringType(), True), StructField('artist_date_created', StringType(), Tru

In [18]:
ds.rdd.count()

106574

In [23]:
ds.take(6)

[Row(album_comments='0', album_date_created='2008-11-26 01:44:45', album_date_released='2009-01-05 00:00:00', album_engineer=None, album_favorites='4', album_id='1', album_information='<p></p>', album_listens='6073', album_producer=None, album_tags='[]', album_title='AWOL - A Way Of Life', album_tracks='7', album_type='Album', artist_active_year_begin='2006-01-01 00:00:00', artist_active_year_end=None, artist_associated_labels=None, artist_bio='<p>A Way Of Life, A Collective of Hip-Hop from NJ...................</p>', artist_comments='0', artist_date_created='2008-11-26 01:42:32', artist_favorites='9', artist_id='1', artist_latitude='40.0583238', artist_location='New Jersey', artist_longitude='-74.4056612', artist_members='Sajje Morocco,Brownbum,ZawidaGod,Custodian of Records,Zooberelli the Don,F.A.H,MadSicka,Damien Omenicci..and a van load more...', artist_name='AWOL', artist_related_projects='The list of past projects is 2 long but every1 and every style from Tabby Bonet 2 M.O.P..Azi

In [24]:
data = ds.rdd.filter(lambda v: v['set_subset'] == 'small')
data.count() # 8000 lines